In [1]:
#imports
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from sklearn.metrics import plot_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import autokeras as ak
#load data
from SB_Test_runner import get_scens_per_dim, get_simulated_data

class newmodel(MLPClassifier):
    def __init__(self, model):
        self.model = model
    def predict(self, X):
        y = self.model.predict(X)
        return np.argmax(y, axis=1)


2022-12-20 20:51:14.393824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 20:51:14.542508: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-20 20:51:15.143170: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda-11.4/lib64
2022-12-20 20:51:15.143273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

## Settings for this experiment

20000 repetitions per class (group of scenes) and 500 samples (runs)

In [2]:
#settings for this experiment
rep = 200
n_samples = 600

## Loading the data

In [3]:
#load data
scenes = get_scens_per_dim()
per_label = {"unif":0, "centre":0, "bounds":0, "gaps/clusters":0, "disc":0}
X = []
y = []
realY = []
for scene in scenes:
    label = scene[0]
    realLabel = f"{label} " + json.dumps(scene[1])
    kwargs = scene[1]
    if (label == "unif"):
        rep1 = 4 * rep
    elif (label in ["trunc_unif", "cauchy", "norm"]):
        rep1 = int(rep / 32)
    elif (label in ["bound_thing","inv_norm", "inv_cauchy"]):
        rep1 = int(rep / 48)
    elif (label in ["clusters","gaps", "part_unif"]):
        rep1 = int(rep / 67)
    elif (label in ["spikes", "shifted_spikes"]):
        rep1 = int(rep / 42)
    data = get_simulated_data(label, rep=rep1, n_samples = n_samples, kwargs=kwargs)
    for r in range(rep1):
        X.append(np.sort(data[:,r]))
    if (label in ["trunc_unif", "cauchy", "norm"]):
        label = "centre"
    elif (label in ["bound_thing","inv_norm", "inv_cauchy"]):
        label = "bounds"
    elif (label in ["gaps", "part_unif", "clusters"]):
        label = "gaps/clusters"
    elif (label in ["spikes", "shifted_spikes"]):
        label = "disc"
    per_label[label] += rep1
    y.extend([label]*rep1)
    realY.extend([realLabel] * rep1)

print(per_label)
X = np.array(X)
int_y, targetnames= pd.factorize(y)
int_real_y, targetnames_real= pd.factorize(realY)

cat_y = to_categorical(int_y)
cat_y_real = to_categorical(int_real_y)

X_train, X_test, y_train, y_test = train_test_split(X, cat_y, test_size=0.2, random_state=42, stratify=int_y)

#expand dims
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

{'unif': 800, 'centre': 192, 'bounds': 192, 'gaps/clusters': 134, 'disc': 168}


## Loading the model from h5 file

In [4]:
#load model
from sklearn.metrics import f1_score

model = tf.keras.models.load_model(f"../BIAS/models/opt_cnn_model-{n_samples}.h5")
#model.summary()
print(
    "Accuracy: {accuracy}".format(
        accuracy = model.evaluate(x=X_test, y=y_test)
    ),
    "f1 score: {f1}".format(
        f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(model.predict(X_test), axis=1), average='macro')
    )
)
#tf.keras.utils.plot_model(model, to_file=f"models/opt_cnn_model-{n_samples}.png")


2022-12-20 20:51:24.790718: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-20 20:51:24.790775: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: neocortex
2022-12-20 20:51:24.790783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: neocortex
2022-12-20 20:51:24.790912: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.42.1
2022-12-20 20:51:24.790943: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.42.1
2022-12-20 20:51:24.790950: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.42.1
2022-12-20 20:51:24.791229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions i

10/10 [==============================] - 0s 18ms/step
Accuracy: [0.21176068484783173, 0.9765100479125977] f1 score: 0.9635308909276988


## Analysis of wrongly predicted cases in the test data

In [5]:
model1 = newmodel(model)
hat_y = model1.predict(X_test)

hat_y_real = model.predict(X_test)

10/10 [==============================] - 0s 18ms/step


## Compute shap values for all test cases

In [6]:
#using shap
import seaborn as sbs
import shap
# select backgroud for shap
background = X_train[np.random.choice(X_train.shape[0], 1000, replace=False)]
# DeepExplainer to explain predictions of the model
explainer = shap.DeepExplainer(model, background)
# compute shap values

#different try
#bg = shap.maskers.Partition(shap.utils.sample(X,100))
#explainer = shap.explainers.Partition(f, bg)

#x_explained = X_test[:100]
#shap_values = explainer.shap_values(x_explained)

/home/neocortex/repos/BIAS/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


In [7]:
#plot shap explanations
from matplotlib.gridspec import GridSpec
def plot_explanation(x, preds, pred_labels, shap_vals_pred, shap_vals_true, prediction, label, filename):
    plt.ioff()
    cmap = sbs.color_palette('coolwarm', as_cmap=True)
    norm = plt.Normalize(vmin=-1*np.max(np.abs(shap_vals_pred)), vmax=np.max(np.abs(shap_vals_pred)))  # 0 and 1 are the defaults, but you can adapt these to fit other uses
    df = pd.DataFrame({"x": x.flatten(), "shap": shap_vals_pred.flatten()})
    palette = {h: cmap(norm(h)) for h in df['shap']}

    #fig, axs = plt.subplots(2, figsize=(8,2))
    fig = plt.figure(figsize=(6,3))
    gs = GridSpec(2, 3, figure=fig)
    ax1 = fig.add_subplot(gs[:, 0]) #prediction labels
    ax1.bar(pred_labels, preds)
    plt.xticks(rotation=30, ha='right')
    ax1.set_title("Prediction probabilities")
    ax1.set_ylim([0,1])

    ax2 = fig.add_subplot(gs[0, 1:])
    ax2.set_title(f"Predicted label: {prediction}")
    sbs.swarmplot(data=df, x="x", hue="shap", palette=palette, ax=ax2, size=3, legend=False)
    ax2.set_xlabel("")
    ax2.set_xlim([0,1])

    cmap = sbs.color_palette('coolwarm', as_cmap=True)
    norm = plt.Normalize(vmin=-1*np.max(np.abs(shap_vals_true)), vmax=np.max(np.abs(shap_vals_true)))  # 0 and 1 are the defaults, but you can adapt these to fit other uses
    df = pd.DataFrame({"x": x.flatten(), "shap": shap_vals_true.flatten()})
    palette = {h: cmap(norm(h)) for h in df['shap']}
    ax3 = fig.add_subplot(gs[1, 1:])
    sbs.swarmplot(data=df, x="x", hue="shap", palette=palette, ax=ax3, size=3, legend=False)
    ax3.set_title(f"True label: {label}")
    ax3.set_xlabel("")
    ax3.set_xlim([0,1])
    
    #sbs.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

test_y = np.argmax(y_test, axis=1)
for i in range(len(hat_y)):
    if hat_y[i] != test_y[i]:
        shap_val = explainer.shap_values(X_test[i:i+1])
        plot_explanation(X_test[i], 
            hat_y_real[i], 
            targetnames,
            shap_val[hat_y[i]][0], 
            shap_val[test_y[i]][0], 
            targetnames[hat_y[i]], 
            targetnames[test_y[i]], 
            f"misclassifications/{n_samples}prediction{i}.png")



`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


22.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
41.8% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
38.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
63.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
38.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
63.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.


## Plot confusion matrix

In [ ]:
#plot confusion matrix

test_y = np.argmax(y_test, axis=1)
fig, ax = plt.subplots(figsize=(14, 14))
plot_confusion_matrix(model1, X_test, test_y, normalize='true', xticks_rotation = 'vertical', display_labels = targetnames, ax=ax) 
plt.savefig(f"models/opt_cnn_model-{n_samples}-confusion.png")
#plt.show()

   9/1000 [..............................] - ETA: 6s 

Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.


1000/1000 [==============================] - 6s 6ms/step


: 